<a href="https://colab.research.google.com/github/pj0620/google-colab-notebooks/blob/main/Minesweeper_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
from scipy.signal import convolve2d
%pip install stable-baselines3[extra]

import numpy as np
import gymnasium as gym
from gymnasium import spaces

!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]

!apt-get install swig cmake ffmpeg
!pip install git+https://github.com/DLR-RM/rl-baselines3-zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.7 MB/s eta 0:00:00
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=d52ba6b47062c451e71836d5b6987244f16f710f562b7710d6ed8fbf087d55f8
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
  Attempting uninstall: shimmy
    Found existing ins

  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.6.1
    Uninstalling AutoROM-0.6.1:
      Successfully uninstalled AutoROM-0.6.1
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 2s (731 kB/s)
Selecting previously unse

## Hyperparameter configurations





Create a file named `dql.yml` with the following contents

```
Minesweeper-v1:
  frame_stack: 1
  policy: 'CnnPolicy'
  n_timesteps: !!float 1e6
  buffer_size: 100000
  learning_rate: !!float 1e-4
  batch_size: 32
  learning_starts: 100000
  target_update_interval: 1000
  train_freq: 4
  gradient_steps: 1
  exploration_fraction: 0.1
  exploration_final_eps: 0.01
  # If True, you need to deactivate handle_timeout_termination
  # in the replay_buffer_kwargs
  optimize_memory_usage: False
```


## Custom gymnasium env

In [25]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from skimage.transform import resize

class MinesweeperEnvironment(gym.Env):
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {"render_modes": ["console"]}

  def __init__(self, board_size=9, total_bombs=10, render_mode="console"):
    super().__init__()
    self.render_mode = render_mode
    self.board_size = board_size
    self.total_bombs = total_bombs

    self.visible = np.zeros((self.board_size, self.board_size), dtype=np.uint8)
    self.set_bombs()
    self.set_values()

    # Define observation space
    # self.observation_space = spaces.Dict({
    #   "visible_vals": spaces.Box(low=0, high=8, shape=(self.board_size, self.board_size), dtype=np.int8),
    #   "visible": spaces.Box(low=0, high=1, shape=(self.board_size, self.board_size), dtype=np.int8)
    # })

    # self.observation_space = spaces.Box(
    #     low=0, high=8, shape=(2 * self.board_size**2,), dtype=np.int8
    # )

    self.observation_space = spaces.Box(
        low=0, high=255, shape=(84, 84, 3), dtype=np.uint8
    )

    # Define action space
    self.action_space = spaces.Discrete(self.board_size**2)

    self.last_action = -1

  def set_bombs(self):
    random.seed(10)
    self.bombs = np.zeros(shape=(self.board_size, self.board_size),  dtype=np.uint8)
    placed_bombs = 0
    while placed_bombs < self.total_bombs:
      i = random.randint(0, self.board_size-1)
      j = random.randint(0, self.board_size-1)

      if self.bombs[i][j] == 0:
        self.bombs[i][j] = 1
        placed_bombs += 1

  def set_values(self):
    KERNAL = np.ones((3, 3))
    self.vals = convolve2d(self.bombs, KERNAL, mode='same').astype(np.uint8)

  def reset(self, seed=None, options=None):
    # Reset the environment to an initial state
    self.visible = np.zeros((self.board_size, self.board_size), dtype=np.uint8)
    self.set_bombs()
    self.set_values()
    self.last_action = -1

    return self.get_state(), {}

  def propogate(self, x: int, y: int):
    if self.bombs[x][y] == 1 or self.visible[x][y] == 1:
      return

    if self.vals[x][y] == 0:
      self.visible[x][y] = 1
      for x_k in [x - 1, x, x + 1]:
        for y_k in [y - 1, y, y + 1]:
          if (x_k, y_k) == (x, y):
            continue

          if x_k < 0 or x_k > self.board_size - 1:
            continue

          if y_k < 0 or y_k > self.board_size - 1:
            continue

          self.propogate(x_k, y_k)

  def step(self, action):
    # Implement the logic for taking a step in the environment
    x = action // self.board_size
    y = action % self.board_size

    start_visible_cells = np.sum(self.visible)

    if self.visible[x][y] == 1:
      start_teminated = bool((self.board_size**2 - start_visible_cells) == self.total_bombs)
      return self.get_state(), -0.3, start_teminated, False, {}
    elif self.bombs[x][y] == 1:
      return self.get_state(), -1, True, True, {}

    self.last_action = action

    if self.vals[x][y] == 0:
      self.propogate(x, y)

    self.visible[x][y] = 1
    end_visible_cells = np.sum(self.visible)
    teminated = bool((self.board_size**2 - end_visible_cells) == self.total_bombs)
    return self.get_state(), 1, teminated, False, {}
    # return self.get_state(), float(end_visible_cells) - float(start_visible_cells), teminated, False, {}

  def get_state(self):
    res = (255 / 8) * np.stack((self.visible * self.vals, self.visible, np.zeros((self.board_size, self.board_size))), axis=2)
    return resize(res, (84, 84, 3), anti_aliasing=False).astype(np.uint8)

  def render(self, mode="console"):
    if self.render_mode != "console":
        raise NotImplementedError("Render mode not supported.")

    # Print the current visible board state
    print("Current Board:")
    for i in range(self.board_size):
        row = ""
        for j in range(self.board_size):
            if self.visible[i][j] == 1:
                # If the cell is visible, show its value (number of adjacent bombs)
                if self.bombs[i][j] == 1:
                  row += f"B "
                else:
                  row += f"{self.vals[i][j]} "
            else:
                # If the cell is hidden, show an asterisk
                row += "* "
        print(row)
    print("\n")


from stable_baselines3.common.env_checker import check_env

env = MinesweeperEnvironment(board_size=10, total_bombs=10)

# Testing env
check_env(env)
for round in range(10):
  obs, info = env.reset()
  for _ in range(10):
      # Random action
      action = env.action_space.sample()
      obs, reward, terminated, truncated, info = env.step(action)
      if terminated:
          obs, info = env.reset()
      # env.render()
  env.reset()

## DQN

In [22]:
from math import log

# 18418
log(0.01) / log(0.99975)

18418 / 100_000

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.18418

In [33]:
import torch as th
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[2]  # Typically 3 for RGB

        # Define a custom CNN architecture
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # Specify input channels as 3 for RGB
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(84 * 84 * 64, 256),  # Adjust the input size to match the flattened output
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 81)  # Output size of 81, with no activation function for linear output
        )
        # self.cnn = nn.Sequential(
        #     nn.Conv2d(n_input_channels, 32, kernel_size=8, stride=4, padding=0),
        #     nn.ReLU(),
        #     nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
        #     nn.ReLU(),
        #     nn.Flatten(),
        # )

        # Calculate the output size of the convolutional layers
        with th.no_grad():
            n_flatten = self.cnn(th.as_tensor(observation_space.sample()[None]).float()).shape[1]

        # Final fully connected layer
        self.linear = nn.Sequential(
            nn.Linear(n_flatten, features_dim),
            nn.ReLU()
        )

    def forward(self, observations):
        return self.linear(self.cnn(observations))


In [34]:
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env

vec_env = make_vec_env(MinesweeperEnvironment, n_envs=1)

# Optimized hyperparameters for DQN
learning_rate = 1e-4           # Learning rate for weight updates
buffer_size = 50000            # Replay buffer size
learning_starts = 2000         # Steps before learning begins
batch_size = 64                # Number of samples per training update
train_freq = (1, 'episode')    # Frequency of training updates
target_update_interval = 5     # Steps between target network updates
exploration_fraction = 0.9     # Fraction of total timesteps for epsilon decay
exploration_final_eps = 0.01   # Final epsilon value after decay
gamma = 0.9                    # Discount factor for future rewards

#  0.99975 ** x = 0.01 => x = log(0.01) / log(0.99975)

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256)  # Output size of the final layer
)

# Instantiate DQN with improved hyperparameters
model = DQN(
    "CnnPolicy",
    vec_env,
    learning_rate=learning_rate,
    buffer_size=buffer_size,
    learning_starts=learning_starts,
    batch_size=batch_size,
    train_freq=train_freq,
    target_update_interval=target_update_interval,
    exploration_fraction=exploration_fraction,
    exploration_final_eps=exploration_final_eps,
    gamma=gamma,
    policy_kwargs=policy_kwargs,  # Use custom network
    verbose=1
)

# Train the model
model.learn(total_timesteps=100_000)

Using cuda device
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.12GB > 1.29GB
  warnings.warn(


Streaming output truncated to the last 5000 lines.
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0497   |
|    n_updates        | 1221     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.52     |
|    ep_rew_mean      | 4.28     |
|    exploration_rate | 0.861    |
| time/               |          |
|    episodes         | 1484     |
|    fps              | 51       |
|    time_elapsed     | 243      |
|    total_timesteps  | 12599    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0758   |
|    n_updates        | 1225     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.66     |
|    ep_rew_mean      | 4.4      |
|    exploration_rate | 0.861    |
| time/               |          |
|    episodes         | 1488     |
|   

KeyboardInterrupt: 

## PPO

In [ ]:
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env

vec_env = make_vec_env(MinesweeperEnvironment, n_envs=1)

# Define hyperparameters for PPO
learning_rate = 1e-4       # Learning rate for PPO
n_steps = 2048             # Number of steps to run for each environment per update
batch_size = 64            # Batch size for each update
n_epochs = 10              # Number of times to train on each batch
gamma = 0.99               # Discount factor
gae_lambda = 0.95          # GAE lambda, for variance reduction in advantage estimation
clip_range = 0.2           # Clip range for PPO, helps with stable training

# Instantiate PPO with custom hyperparameters
model = PPO(
    "CnnPolicy",           # Policy type, can try "CnnPolicy" for image-based inputs
    vec_env,
    learning_rate=learning_rate,
    n_steps=n_steps,
    batch_size=batch_size,
    n_epochs=n_epochs,
    gamma=gamma,
    gae_lambda=gae_lambda,
    clip_range=clip_range,
    verbose=1               # Verbose output
)

# Train the model
model.learn(total_timesteps=100000)

Using cuda device
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.63e+03 |
|    ep_rew_mean     | -9.6e+05 |
| time/              |          |
|    fps             | 226      |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.63e+03  |
|    ep_rew_mean          | -9.6e+05  |
| time/                   |           |
|    fps                  | 207       |
|    iterations           | 2         |
|    time_elapsed         | 19        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 25.478064 |
|    clip_fraction        | 0.66      |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.98     |
|    explained_variance   | -6.08e-06 |
|

## Evaluation

In [27]:
# Test the trained agent
# using the vecenv
obs = vec_env.reset()
n_steps = 100
for step in range(n_steps):
    action, _ = model.predict(obs, deterministic=False)
    # print(f"Step {step + 1}")
    x = int(action[0]) // 10
    y = int(action[0]) % 10
    print("Action: ", (x, y))
    obs, reward, done, info = vec_env.step(action)
    print("reward=", reward, "done=", done)
    vec_env.render()
    if done:
        # Note that the VecEnv resets automatically
        # when a done signal is encountered
        print("Goal reached!", "reward=", reward)
        break

Action:  (6, 5)
reward= [1.] done= [False]
Current Board:
* * * * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * 0 * * * * * * 
* * 0 * * * * * * 


Action:  (2, 2)
reward= [1.] done= [False]
Current Board:
* * * * * * * * * 
* * * * * * * * * 
* * * * 2 * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * 0 * * * * * * 
* * 0 * * * * * * 


Action:  (3, 5)
reward= [1.] done= [False]
Current Board:
* * * * * * * * * 
* * * * * * * * * 
* * * * 2 * * * * 
* * * * * * * * 1 
* * * * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * 0 * * * * * * 
* * 0 * * * * * * 


Action:  (3, 8)
reward= [1.] done= [False]
Current Board:
* * * * * * * * * 
* * * * * * * * * 
* * * * 2 * * * * 
* * * * * * * * 1 
* * 1 * * * * * * 
* * * * * * * * * 
* * * * * * * * * 
* * 0 * * * * * * 
* * 0 * * * * * * 


Action:  (1, 0)
reward= [1.] done= [False]
Current Board:
* * * * * * * * * 

## Old RL Zoo code

In [ ]:
# from rl_zoo3.train import train
# from gym.envs.registration import register

# register(
#     id='Minesweeper-v1',
#     entry_point='msenv:MinesweeperEnvironment',  # Update '__main__' to the module name if this is not in your main script
#     max_episode_steps=100,  # Adjust based on expected game length
# )

# import gym
# print([k for k in gym.envs.registry.keys() if "Minesweeper" in k])

# !python -m rl_zoo3.train --algo dqn --env Minesweeper-v1 -f logs/ -c dqn.yml
